In [ ]:
import dash
from dash import dcc, html
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# Simulated Data for Demo (Improved for Realism)
dates = pd.date_range(start='2024-03-01', periods=30, freq='D')
open_prices = np.random.randn(30) * 2 + 150
close_prices = open_prices + np.random.randn(30) * 5  # Adding more variation
high_prices = np.maximum(open_prices, close_prices) + np.random.rand(30) * 3  # High > Close/Open
low_prices = np.minimum(open_prices, close_prices) - np.random.rand(30) * 3  # Low < Close/Open
volumes = np.random.randint(1000, 5000, size=30)
ratings = np.random.uniform(3.5, 5, size=30)

# Create DataFrame
data = pd.DataFrame({
    'Date': dates,
    'Open': open_prices,
    'Close': close_prices,
    'High': high_prices,
    'Low': low_prices,
    'Volume': volumes,
    'Ratings': ratings
})
data.set_index('Date', inplace=True)

# Moving Averages (MA10, MA20, and Volume MA)
data['MA10'] = data['Close'].rolling(window=10).mean()
data['MA20'] = data['Close'].rolling(window=20).mean()
data['Volume_MA'] = data['Volume'].rolling(window=10).mean()

# RSI (Relative Strength Index) Calculation
delta = data['Close'].diff()
gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
rs = gain / loss
data['RSI'] = 100 - (100 / (1 + rs))

# Bollinger Bands
data['Bollinger_Mid'] = data['Close'].rolling(window=20).mean()
data['Bollinger_Upper'] = data['Bollinger_Mid'] + 2 * data['Close'].rolling(window=20).std()
data['Bollinger_Lower'] = data['Bollinger_Mid'] - 2 * data['Close'].rolling(window=20).std()

# MACD (Moving Average Convergence Divergence)
data['MACD_Line'] = data['Close'].ewm(span=12, adjust=False).mean() - data['Close'].ewm(span=26, adjust=False).mean()
data['MACD_Signal'] = data['MACD_Line'].ewm(span=9, adjust=False).mean()

# Stock Price Forecast using Linear Regression
X = np.array(range(len(data))).reshape(-1, 1)  # Feature (time)
y = data['Close'].values  # Target (price)
model = LinearRegression().fit(X, y)
data['Forecast'] = model.predict(X)

# Initialize Dash App
app = dash.Dash(__name__)

# Define the layout
app.layout = html.Div(children=[
    html.H1("Advanced Stock Market Dashboard", style={'text-align': 'center'}),

    dcc.Dropdown(
        id='section-dropdown',
        options=[
            {'label': 'Stock Price & Moving Averages', 'value': 'price'},
            {'label': 'Trading Volume', 'value': 'volume'},
            {'label': 'Customer Sentiment Ratings', 'value': 'ratings'},
            {'label': 'Candlestick Chart', 'value': 'candlestick'},
            {'label': 'RSI Indicator', 'value': 'rsi'},
            {'label': 'Bollinger Bands', 'value': 'bollinger'},
            {'label': 'MACD', 'value': 'macd'},
            {'label': 'Stock Price Forecast', 'value': 'forecast'}
        ],
        value='price',
        style={'width': '50%', 'margin': '0 auto'}
    ),
    dcc.DatePickerRange(
        id='date-picker-range',
        start_date=data.index.min().date(),
        end_date=data.index.max().date(),
        display_format='YYYY-MM-DD',
        style={'margin': '20px 0', 'width': '50%', 'margin-left': 'auto', 'margin-right': 'auto'}
    ),

    dcc.RadioItems(
        id='theme-radio',
        options=[
            {'label': 'Light', 'value': 'plotly'},
            {'label': 'Dark', 'value': 'plotly_dark'}
        ],
        value='plotly_dark',
        style={'text-align': 'center', 'margin': '10px'}
    ),
    html.Div(id='chart-container', children=[
        dcc.Graph(id='main-chart')
    ])
])
@app.callback(
    dash.dependencies.Output('main-chart', 'figure'),
    [dash.dependencies.Input('section-dropdown', 'value'),
     dash.dependencies.Input('date-picker-range', 'start_date'),
     dash.dependencies.Input('date-picker-range', 'end_date'),
     dash.dependencies.Input('theme-radio', 'value')]
)
def update_chart(selected_section, start_date, end_date, selected_theme):
    filtered_data = data[(data.index >= start_date) & (data.index <= end_date)]

    fig = go.Figure()

    if selected_section == 'price':
        fig.add_trace(go.Scatter(x=filtered_data.index, y=filtered_data['Close'], mode='lines', name='Stock Price', line=dict(color='blue')))
        fig.add_trace(go.Scatter(x=filtered_data.index, y=filtered_data['MA10'], mode='lines', name='10-Day MA', line=dict(color='orange', dash='dash')))
        fig.add_trace(go.Scatter(x=filtered_data.index, y=filtered_data['MA20'], mode='lines', name='20-Day MA', line=dict(color='red', dash='dot')))
        fig.update_layout(title="Stock Price & Moving Averages", template=selected_theme)

    elif selected_section == 'candlestick':
        fig = go.Figure(data=[go.Candlestick(
            x=filtered_data.index,
            open=filtered_data['Open'],
            high=filtered_data['High'],
            low=filtered_data['Low'],
            close=filtered_data['Close'],
            increasing_line_color='green',
            decreasing_line_color='red'
        )])
        fig.update_layout(title="Candlestick Chart", template=selected_theme)

    elif selected_section == 'rsi':
        fig.add_trace(go.Scatter(x=filtered_data.index, y=filtered_data['RSI'], mode='lines', name='RSI', line=dict(color='purple')))
        fig.update_layout(title="RSI Indicator", template=selected_theme)

    elif selected_section == 'bollinger':
        fig.add_trace(go.Scatter(x=filtered_data.index, y=filtered_data['Close'], mode='lines', name='Price', line=dict(color='blue')))
        fig.add_trace(go.Scatter(x=filtered_data.index, y=filtered_data['Bollinger_Upper'], mode='lines', name='Bollinger Upper', line=dict(color='red', dash='dot')))
        fig.add_trace(go.Scatter(x=filtered_data.index, y=filtered_data['Bollinger_Lower'], mode='lines', name='Bollinger Lower', line=dict(color='red', dash='dot')))
        fig.update_layout(title="Bollinger Bands", template=selected_theme)

    elif selected_section == 'macd':
        fig.add_trace(go.Scatter(x=filtered_data.index, y=filtered_data['MACD_Line'], mode='lines', name='MACD Line', line=dict(color='blue')))
        fig.add_trace(go.Scatter(x=filtered_data.index, y=filtered_data['MACD_Signal'], mode='lines', name='MACD Signal', line=dict(color='red', dash='dot')))
        fig.update_layout(title="MACD", template=selected_theme)

    elif selected_section == 'forecast':
        fig.add_trace(go.Scatter(x=filtered_data.index, y=filtered_data['Close'], mode='lines', name='Actual Prices', line=dict(color='blue')))
        fig.add_trace(go.Scatter(x=filtered_data.index, y=filtered_data['Forecast'], mode='lines', name='Forecast Prices', line=dict(color='green', dash='dot')))
        fig.update_layout(title="Stock Price Forecast", template=selected_theme)

    return fig

# Run the app
if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
import yfinance as yf
import dash
from dash import dcc, html, Input, Output, callback
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
import pandas as pd

# List of stocks
STOCKS = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'NVDA', 'META', 'NFLX', 'BRK-B', 'JPM', 'V', 'MA', 'DIS', 'PYPL', 'ADBE']

def fetch_stock_data(stock, period='5y'):
    df = yf.download(stock, period=period)
    df.reset_index(inplace=True)
    return df

def get_financial_statements(stock):
    try:
        ticker = yf.Ticker(stock)
        income_stmt = ticker.financials
        balance_sheet = ticker.balance_sheet
        cash_flow = ticker.cashflow
        return income_stmt, balance_sheet, cash_flow
    except:
        return pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

def calculate_indicators(df):
    df['SMA'] = df['Close'].rolling(window=50).mean()
    df['EMA'] = df['Close'].ewm(span=50, adjust=False).mean()
    df['RSI'] = 100 - (100 / (1 + df['Close'].pct_change().rolling(14).mean()))
    return df

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.CYBORG])

theme_switch = dbc.Switch(id='theme-switch', value=False, label='Dark Mode')

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col(html.H2("Stock Dashboard"), width=6),
        dbc.Col(theme_switch, width=2)
    ], className="mb-3"),
    dcc.Dropdown(
        id='stock-selector',
        options=[{'label': stock, 'value': stock} for stock in STOCKS],
        value='AAPL',
        clearable=False,
        className="mb-3"
    ),
    dcc.Graph(id='candlestick-chart'),
    dcc.Graph(id='stock-indicators'),
    dbc.Tabs([
        dbc.Tab(dcc.Graph(id='volume-chart'), label='Volume'),
        dbc.Tab(html.Div(id='financial-statement'), label='Financial Statements')
    ], className="mt-3"),
    html.Div(id='trade-section', className="mt-3", children=[
        dbc.Button("Buy", id="buy-button", color="success", className="me-2"),
        dbc.Button("Sell", id="sell-button", color="danger"),
    ]),
    dcc.Interval(id='interval-update', interval=60*1000, n_intervals=0)
], fluid=True)

@callback(
    [Output('candlestick-chart', 'figure'),
     Output('stock-indicators', 'figure'),
     Output('volume-chart', 'figure'),
     Output('financial-statement', 'children')],
    [Input('stock-selector', 'value'),
     Input('interval-update', 'n_intervals')]
)
def update_stock_data(selected_stock, n_intervals):
    df = fetch_stock_data(selected_stock)
    df = calculate_indicators(df)
    income_stmt, balance_sheet, cash_flow = get_financial_statements(selected_stock)
    
    fig_candlestick = go.Figure()
    fig_candlestick.add_trace(go.Candlestick(
        x=df['Date'], open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'],
        increasing_line_color='green', decreasing_line_color='red'
    ))
    fig_candlestick.update_layout(
        title="Candlestick Chart",
        template='plotly_dark',
        paper_bgcolor='#1e1e1e',
        plot_bgcolor='#1e1e1e',
        font=dict(color='white')
    )
    
    fig_indicators = go.Figure()
    fig_indicators.add_trace(go.Scatter(x=df['Date'], y=df['SMA'], mode='lines', name='SMA', line=dict(color='blue')))
    fig_indicators.add_trace(go.Scatter(x=df['Date'], y=df['EMA'], mode='lines', name='EMA', line=dict(color='orange')))
    fig_indicators.add_trace(go.Scatter(x=df['Date'], y=df['RSI'], mode='lines', name='RSI', line=dict(color='purple')))
    fig_indicators.update_layout(
        title="Stock Indicators",
        template='plotly_dark',
        paper_bgcolor='#1e1e1e',
        plot_bgcolor='#1e1e1e',
        font=dict(color='white')
    )
    
    fig_volume = go.Figure()
    fig_volume.add_trace(go.Bar(x=df['Date'], y=df['Volume'], name='Volume', marker_color='cyan'))
    fig_volume.update_layout(
        title="Trading Volume",
        template='plotly_dark',
        paper_bgcolor='#1e1e1e',
        plot_bgcolor='#1e1e1e',
        font=dict(color='white')
    )
    
    financials = html.Div([
        html.H5("Income Statement", style={'color': 'white'}), html.Pre(income_stmt.to_string(), style={'color': 'white'}),
        html.H5("Balance Sheet", style={'color': 'white'}), html.Pre(balance_sheet.to_string(), style={'color': 'white'}),
        html.H5("Cash Flow Statement", style={'color': 'white'}), html.Pre(cash_flow.to_string(), style={'color': 'white'})
    ], style={'backgroundColor': '#1e1e1e', 'padding': '10px'})
    
    return fig_candlestick, fig_indicators, fig_volume, financials

@callback(
    Output('theme-switch', 'label'),
    Input('theme-switch', 'value')
)
def toggle_theme(is_dark):
    return "Dark Mode" if is_dark else "Light Mode"

if __name__ == '__main__':
    app.run(debug=True)
